In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io

from smt.sampling_methods import LHS
from scipy.io import savemat

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/tanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [6]:
def true_1D_2(x): #True function for 1D_1 dy/dx = cos(0.01*x) BC1: y(0)=0; x \in [-100,100]
    y = 100*np.sin(0.01*x)
    return y

In [7]:
loss_thresh = 0.005

x = np.linspace(-600,600,5000).reshape(-1,1)
ysol = true_1D_2(x)

bc1_x = np.array(0).reshape(-1,1) 
bc1_y = np.array(0).reshape(-1,1)
x_bc1_train = torch.from_numpy(bc1_x).float().to(device)
y_bc1_train = torch.from_numpy(bc1_y).float().to(device)

 
x_test = x.reshape(-1,1)
x_test_tensor = torch.from_numpy(x_test).float().to(device)

y_true = true_1D_2(x_test)
y_true_norm = np.linalg.norm(y_true,2)

# Domain bounds
lb = np.array(x[0]) 
ub = np.array(x[-1]) 

#torch.autograd.set_detect_anomaly(True)

In [8]:
def colloc_pts(N_f,seed):

  #Collocation Points
  # Latin Hypercube sampling for collocation points 
  # N_f sets of tuples(x,y)
  x01 = np.array([[0.0, 1.0]])
  sampling = LHS(xlimits=x01,random_state =seed)

  x_coll_train = lb + (ub-lb)*sampling(N_f)
  x_coll_train = np.vstack((x_coll_train, bc1_x)) # append training points to collocation points 

  return x_coll_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
    
        self.activation = nn.Tanh()
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data) 
             
      
              
    'forward pass'
    def forward(self,x):
        if torch.is_tensor(x) != True:         
            x = torch.from_numpy(x)                
        
        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)
                      
        #preprocessing input 
        x = (x - l_b)/(u_b - l_b) #feature scaling
        
        #convert to float
        a = x.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) 
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC1(self,x,y):
                
        loss_bc1 = self.loss_function(self.forward(x), y)
                
        return loss_bc1
    
    def loss_PDE(self, x_coll,f_hat):
             
        g = x_coll.clone()             
        g.requires_grad = True
  
        y = self.forward(g) 

        y_x = autograd.grad(y,g,torch.ones([x_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]

        dy_dx = y_x[:,[0]]
        
        f = dy_dx - torch.cos(0.01*g)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    
    def loss(self,x_bc1,y_bc1,x_coll,f_hat):

        loss_bc1 = self.loss_BC1(x_bc1,y_bc1)
        loss_f = self.loss_PDE(x_coll,f_hat)
        
        loss_val = loss_bc1 + 100*loss_f
        
        return loss_val
     
    
    def test(self):
        y_pred = self.forward(x_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()

        return y_pred

    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(seed):
    x_coll_np_array = colloc_pts(N_f,seed*123)
    x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(x_bc1_train,y_bc1_train,x_coll_train,f_hat)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
   
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep):
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time()
    thresh_flag = 0

    x_coll_np_array = colloc_pts(N_f,123)
    x_coll = torch.from_numpy(x_coll_np_array).float().to(device)

    f_hat = torch.zeros(x_coll.shape[0],1).to(device)

    for i in range(max_iter):
        train_step(i)        
    
        loss_np = PINN.loss(x_bc1_train,y_bc1_train,x_coll,f_hat).cpu().detach().numpy()
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    elapsed_time[rep] = time.time() - start_time
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:
for tune_reps in range(5):
  label = "1D_FODE_tanh_tune"+str(tune_reps)  
  max_reps = 10
  max_iter = 100

  train_loss_full = []
  test_mse_full = []
  test_re_full = []

  elapsed_time= np.zeros((max_reps,1))
  time_threshold = np.empty((max_reps,1))
  time_threshold[:] = np.nan
  epoch_threshold = max_iter*np.ones((max_reps,1))


  for reps in range(max_reps):   
      train_loss = []
      test_mse_loss = []
      test_re_loss = []   

      
      torch.manual_seed(reps*36)
      N_f = 10000 #Total number of collocation points
    
      layers = np.array([1,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
      PINN = Sequentialmodel(layers)
    
      PINN.to(device)

      'Neural Network Summary'
      print(PINN)

      params = list(PINN.parameters())
      
      optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-6, 
                                tolerance_change = 1e-6, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')
    

      
      train_model(max_iter,reps)

      
      torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
      train_loss_full.append(train_loss)
      test_mse_full.append(test_mse_loss)
      test_re_full.append(test_re_loss)
      
      
      print('Training time: %.2f' % (elapsed_time[reps]))

  mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "label": label}
  savemat(label+'.mat', mdic)

Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 47.768353 Test MSE 5220.105863394234 Test RE 0.9997532392787468
1 Train Loss 47.76812 Test MSE 5219.8305264904575 Test RE 0.9997268727070199
2 Train Loss 47.767845 Test MSE 5219.027356350406 Test RE 0.9996499562575324
3 Train Loss 47.754787 Test MSE 5174.039218655153 Test RE 0.995332128

97 Train Loss 27.326359 Test MSE 11234.040242414449 Test RE 1.4666319858468215
98 Train Loss 27.326359 Test MSE 11234.040242414449 Test RE 1.4666319858468215
99 Train Loss 27.326359 Test MSE 11234.040242414449 Test RE 1.4666319858468215
Training time: 31.28
Training time: 31.28
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 47.76849

94 Train Loss 12.751297 Test MSE 6541.316756318631 Test RE 1.1191434432029481
95 Train Loss 12.652051 Test MSE 6727.820361266276 Test RE 1.1349856173160506
96 Train Loss 12.607227 Test MSE 6829.567591965474 Test RE 1.1435358081960225
97 Train Loss 12.556284 Test MSE 6853.400357444191 Test RE 1.145529337463977
98 Train Loss 12.483355 Test MSE 6900.011212000621 Test RE 1.1494181824854632
99 Train Loss 12.302089 Test MSE 6900.5755994715855 Test RE 1.1494651899421342
Training time: 44.73
Training time: 44.73
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_f

91 Train Loss 23.941534 Test MSE 6205.943079803036 Test RE 1.090076704540904
92 Train Loss 23.941534 Test MSE 6205.943079803036 Test RE 1.090076704540904
93 Train Loss 23.941534 Test MSE 6205.943079803036 Test RE 1.090076704540904
94 Train Loss 23.941534 Test MSE 6205.943079803036 Test RE 1.090076704540904
95 Train Loss 23.941534 Test MSE 6205.943079803036 Test RE 1.090076704540904
96 Train Loss 23.941534 Test MSE 6205.943120786554 Test RE 1.0900767081402911
97 Train Loss 23.941534 Test MSE 6205.943120786554 Test RE 1.0900767081402911
98 Train Loss 23.941534 Test MSE 6205.943120786554 Test RE 1.0900767081402911
99 Train Loss 23.941534 Test MSE 6205.943120786554 Test RE 1.0900767081402911
Training time: 39.54
Training time: 39.54
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=1, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50,

87 Train Loss 5.6655455 Test MSE 1048.3424249231873 Test RE 0.4480275758372697
88 Train Loss 5.6536736 Test MSE 1094.0348363791893 Test RE 0.4576871719672252
89 Train Loss 5.636524 Test MSE 1103.8532776765778 Test RE 0.45973634677411235
90 Train Loss 5.625115 Test MSE 1089.8443711365164 Test RE 0.456809794851271
91 Train Loss 5.622587 Test MSE 1090.7069944380573 Test RE 0.4569905439747523
92 Train Loss 5.618053 Test MSE 1088.6586771553434 Test RE 0.45656123458112857
93 Train Loss 5.612553 Test MSE 1071.2985558233893 Test RE 0.45290636525745825
94 Train Loss 5.611529 Test MSE 1064.3047899355702 Test RE 0.4514255886561702
95 Train Loss 5.6104326 Test MSE 1057.6595518247757 Test RE 0.4500140908562239
96 Train Loss 5.610236 Test MSE 1058.4628738834253 Test RE 0.45018495757216764
97 Train Loss 5.610236 Test MSE 1058.4628738834253 Test RE 0.45018495757216764
98 Train Loss 5.610234 Test MSE 1058.4638021248672 Test RE 0.45018515497174244
99 Train Loss 5.609895 Test MSE 1061.185105333673 Test R

...

0 Train Loss 47.73836 Test MSE 5159.785795960272 Test RE 0.9939602142048287
1 Train Loss 47.714077 Test MSE 5092.171118947609 Test RE 0.9874262294042616
2 Train Loss 47.714077 Test MSE 5092.1710985499385 Test RE 0.987426227426599
3 Train Loss 47.637928 Test MSE 5004.1403709048645 Test RE 0.9788539697802152
4 Train Loss 47.603718 Test MSE 4952.048452262594 Test RE 0.9737458221031218
5 Train Loss 47.483997 Test MSE 4825.7110203141465 Test RE 0.9612443947782153
6 Train Loss 46.13164 Test MSE 4776.21786705038 Test RE 0.9563023634894048
7 Train Loss 44.235855 Test MSE 4802.805744452829 Test RE 0.9589604042149552
8 Train Loss 37.359215 Test MSE 4222.013126326932 Test RE 0.8991102615132225
9 Train Loss 35.42304 Test MSE 4307.52052214835 Test RE 0.9081693532862972
10 Train Loss 32.967896 Test MSE 4491.787275171967 Test RE 0.9273907395191688
11 Train Loss 31.445593 Test MSE 4600.635541952069 Test RE 0.9385600831825492
12 Train Loss 26.898378 Test MSE 4261.405585344708 Test RE 0.9032949870367365

0 Train Loss 47.76583 Test MSE 5214.98781471073 Test RE 0.9992630154512607
1 Train Loss 47.742302 Test MSE 5162.3241380639565 Test RE 0.9942046721087143
2 Train Loss 47.69255 Test MSE 5068.744492109965 Test RE 0.9851522747871525
3 Train Loss 47.298847 Test MSE 4909.220542268585 Test RE 0.969525946299372
4 Train Loss 46.911057 Test MSE 4862.337782899487 Test RE 0.9648853833618384
5 Train Loss 45.416103 Test MSE 4974.459778173365 Test RE 0.9759467597614553
6 Train Loss 37.406406 Test MSE 4388.3973319208735 Test RE 0.9166554731257424
7 Train Loss 35.60563 Test MSE 4439.170200786083 Test RE 0.9219429823962798
8 Train Loss 34.07308 Test MSE 4309.674775746945 Test RE 0.9083964192444437
9 Train Loss 33.02846 Test MSE 4381.337225690991 Test RE 0.9159178131608612
10 Train Loss 31.7486 Test MSE 4477.082672865965 Test RE 0.9258715123003154
11 Train Loss 31.044117 Test MSE 4530.981063312852 Test RE 0.9314279977489548
12 Train Loss 30.75769 Test MSE 4602.943099289895 Test RE 0.9387954321555998
13 T

0 Train Loss 47.751633 Test MSE 5181.869014985734 Test RE 0.9960849541145849
1 Train Loss 47.71429 Test MSE 5087.182608926411 Test RE 0.986942448272926
2 Train Loss 47.6972 Test MSE 5050.805382790205 Test RE 0.9834074227039887
3 Train Loss 47.66997 Test MSE 5001.76657452839 Test RE 0.9786217744919964
4 Train Loss 47.652348 Test MSE 4953.37899420787 Test RE 0.973876628844691
5 Train Loss 47.62155 Test MSE 4885.037676549369 Test RE 0.9671350513901202
6 Train Loss 47.607903 Test MSE 4826.584731272694 Test RE 0.961331409080053
7 Train Loss 47.59375 Test MSE 4794.399872207773 Test RE 0.9581208502946245
8 Train Loss 47.56113 Test MSE 4704.162587786598 Test RE 0.9490614355497924
9 Train Loss 47.493576 Test MSE 4626.660611037221 Test RE 0.9412109826623615
10 Train Loss 47.399414 Test MSE 4525.729789285038 Test RE 0.930888092418146
11 Train Loss 47.331947 Test MSE 4485.334125483996 Test RE 0.9267243297040348
12 Train Loss 47.231884 Test MSE 4474.137383089818 Test RE 0.9255669156762467
13 Train 

0 Train Loss 47.744625 Test MSE 5163.296482666238 Test RE 0.9942982989329904
1 Train Loss 47.72692 Test MSE 5140.856801341405 Test RE 0.9921353366120979
2 Train Loss 47.61168 Test MSE 5051.423681470567 Test RE 0.9834676131942528
3 Train Loss 42.8095 Test MSE 4710.22470383142 Test RE 0.9496727524535138
4 Train Loss 39.690334 Test MSE 4469.416237361765 Test RE 0.9250784539606656
5 Train Loss 37.853188 Test MSE 4384.505510963143 Test RE 0.9162489180662677
6 Train Loss 35.900105 Test MSE 4389.558024955104 Test RE 0.9167766888224448
7 Train Loss 35.59297 Test MSE 4334.369322795839 Test RE 0.9109952690887891
8 Train Loss 34.86344 Test MSE 4324.913724417946 Test RE 0.9100010404217472
9 Train Loss 34.051926 Test MSE 4276.3843735068895 Test RE 0.9048811298199358
10 Train Loss 33.71763 Test MSE 4307.643915388633 Test RE 0.9081823609035715
11 Train Loss 33.01332 Test MSE 4278.614720708336 Test RE 0.9051170693239744
12 Train Loss 32.438885 Test MSE 4277.8341637481435 Test RE 0.9050345043234799
13 

0 Train Loss 47.74756 Test MSE 5173.162802658763 Test RE 0.9952478263601927
1 Train Loss 47.701866 Test MSE 5098.5612423312705 Test RE 0.9880455916576739
2 Train Loss 47.43044 Test MSE 4900.245232464416 Test RE 0.968639270240231
3 Train Loss 46.979095 Test MSE 4831.307689837614 Test RE 0.9618016399468081
4 Train Loss 46.08977 Test MSE 5147.024356050879 Test RE 0.9927302972410346
5 Train Loss 37.927357 Test MSE 4629.265242187205 Test RE 0.9414758780935707
6 Train Loss 35.55877 Test MSE 4485.511774929395 Test RE 0.9267426817853404
7 Train Loss 34.420235 Test MSE 4523.035491178015 Test RE 0.9306109588355115
8 Train Loss 32.40288 Test MSE 4748.233803296734 Test RE 0.9534967397615344
9 Train Loss 29.678894 Test MSE 5123.584013964463 Test RE 0.9904671942929559
10 Train Loss 28.031248 Test MSE 6098.612906591757 Test RE 1.080609295214234
11 Train Loss 27.558409 Test MSE 6574.914127922804 Test RE 1.1220138225695047
12 Train Loss 27.158045 Test MSE 6902.07685118339 Test RE 1.1495902188411276
13 

In [14]:
import scipy.io as sio
import numpy as np

In [15]:
for tune_reps in range(5):
    label = "1D_FODE_tanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(np.mean(re[:,-1]))

0.9479122606253909
0.6029990233605438
0.45482597310115896
0.6878933591732673
0.9690876820687466
